In [7]:
def insidepage(url_inside):
    import bs4
    from urllib.request import urlopen as req
    import re
    import csv
    import math
    url = url_inside
    #     print(url)
    client=req(url)
    var1=client.read()
    client.close()
    page_soup = bs4.BeautifulSoup(var1,"html.parser")
    address=page_soup.findAll("div",{"class":"PropertyDetailsPage__propertyDetailsContainer"})
#     global floor_area
    global Area_code
    global long_lat_tmp
    global longitude
    for address1 in address:
        r1=address1.findAll("div",{"class":"PropertyMainInformation__eircode"})  
    #     print(r1)
        if r1==[]:
            Area_code=str('D00')
        else:
            r=r1[0].text.strip()
            r=r.split()
            Area_code=str(r[1])
#         print(Area_code)
    
        r2=address1.script.text.strip()
        r3=re.search(r"\[-.?\d.*,*\d\],",r2,re.I|re.S)
        if r3:
            long_lat_tmp=r3.group()
            long_lat_tmp.split()
        long_lat=long_lat_tmp.replace("[","").replace("]","").replace(",","")
        long_lat1=long_lat.split()
        longitude=long_lat1[0]
        latitude=long_lat1[1]
    #     print(longitude)
    #     print(latitude)

#         r2=address1.findAll("div",{"class":"PropertyOverview__propertyOverviewDetails"})
#     #     print(r2)
#         r3=r2[0].text.strip()
#         r4=re.search(r"\d.*\d ",r3,re.I|re.S)
#         if r4:
#             floor_area=r4.group()
#     #     print(r4)
    
        description=address1.p.text
        r2=address1.findAll("div",{"class":"PropertyStatistics__iconsContainer"})
        r3=r2[0].text.strip()
        Entered_tmp=re.search(r"\d\d.\d\d.\d\d\d\d",r3,re.I|re.S)
        if Entered_tmp:
            Entered=Entered_tmp.group()
        Views_tmp=re.search(r"[0-9]{0,6}$",r3,re.I|re.S)
        if Views_tmp:
            Views=Views_tmp.group()
            Views_tmp
        return(Area_code,longitude,latitude,Entered,Views)

In [8]:
import bs4
from urllib.request import urlopen as req
import re
import csv
import math
i=60
f=open("C:/Users/Akhilesh/Desktop/daftrent_May.csv","a+")
header="Location,Area,Price(in Euros),House_type,Beds,Bathroom,Area_code,longitude,latitude,Entered,Views\n"
f.write(header)
while(i<=2580):
    url = 'https://www.daft.ie/dublin-city/residential-property-for-rent/?s%5Badvanced%5D=1&s%5Bignored_agents%5D%5B0%5D=1551&searchSource=rental&offset='+str(i)
#     print(url)
    client=req(url)
    var1=client.read()
    client.close()
    page_soup = bs4.BeautifulSoup(var1,"html.parser")
    address=page_soup.findAll("div",{"class":"FeaturedCardPropertyInformation__detailsContainer"})
    if address==[]:
        address=page_soup.findAll("div",{"class":"StandardPropertyInfo__detailsCopyContainer"})
# address1=address[0].findAll("div",{"class":"search_result_title_box"})
# address1[0].a.text.strip()
# info1=address[0].findAll("div",{"class":"info-box"})
# info1[0].strong    
    for address1 in address:
            r1=address1.findAll("div",{"class":"PropertyInformationCommonStyles__addressCopy calculate-truncation-plugin"})  
            r=r1[0].a.text.strip()
            r2=address1.findAll("div",{"class":"PropertyInformationCommonStyles__quickPropertyDetailsContainer"})  
            r3=r2[0].a.text.strip()
            location=r
#             print(r)
            address2=re.search(r"[C|o]{2}[.]{1} .Dublin|Dublin [0-9][0-9]?",r,re.I|re.S)
            if address2:
                area_tmp=address2.group()
                area=area_tmp.strip()
#             print(area)
            address2=address1.findAll("div",{"class":"PropertyInformationCommonStyles__propertyPrice"})
            price_tmp = address2[0].strong.text
            price=price_tmp.replace("€","").replace(",","")
            price_tmp=re.search(r".*Per week",price,re.I|re.S)
            price_tmp1=re.search(r".*Per month",price,re.I|re.S)
            if price_tmp:
                price1=int(price_tmp.group().replace("Per week",""))/7
                price=math.ceil(price1*30)
            elif price_tmp1:
                price=int(price_tmp1.group().replace("Per month",""))
            housedescription_tmp=r3
            housedescription=housedescription_tmp.replace("\n","").strip()
    #     print(housedescription)
            housedescription=housedescription.replace("|                     ","|")
            beds_bath_tmp=re.search(r"([0-9])([0-9])",housedescription,re.I|re.S)
            if beds_bath_tmp:
                beds=beds_bath_tmp.group(1)
                bathroom=beds_bath_tmp.group(2)
            housetype_tmp=re.search(r"[A-Z][a-z].*",housedescription,re.I|re.S)
            if housetype_tmp:
                house_type=housetype_tmp.group()
            list=[]
            r1=address1.findAll("div",{"class":"PropertyInformationCommonStyles__addressCopy calculate-truncation-plugin"})
            for r in r1[0].findAll('a'):
                list.append(r.get('href'))
                inside_url='https://www.daft.ie' + str(list[0])
            Area_code,longitude1,latitude,Entered,Views=insidepage(inside_url)
            f.write(location.replace(",","||") + ","+ area + "," + str(price) + "," + house_type 
                + "," + beds +"," + bathroom + "," + str(Area_code) + "," + longitude1 + "," + latitude + "," 
                    + Entered + "," + Views + "\n")
#             print(location)
#             print(housedescription)
#             print(area)
#             print(price)
#             print(house_type)
#             print(beds)
#             print(bathroom)
#             print(longitude)
#             print(latitude)
#             print(Area_code)
#             print(Entered)
#             print(Views)
    i=i+20
f.close()
        
    #     price=info1.strong.text.strip()
    #     print(info1[0].strong.text)
    #     address2=re.search(r"[A-Z|a-z]{2}[.]{1} .*Dublin.*| Dublin.*",location,re.I|re.S)
    #     if address2:
    #         area=address2.group()
    # #     print(area)
    # #         print(r2)
    # #         location=address1.a.text()
    #     f.write(location.replace(",","||") + "," + area + "\n")
    # # f.seek(0,0)
    # # for price in price_bed_bath:
    # #     houseprice=price.strong.text.strip()
    # # #     print(houseprice)
    # #     f.write("," + "," + houseprice.replace(",","") + "\n")
    # #     f.write("\n")
    # f.close()

60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
